<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/2023_0927_2020_hrs_Yates_near_finish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import csv
import glob
import os

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    @staticmethod
    def get_standard_name(file_path):
        file_name = file_path.split('/')[-1]
        if '.' in file_name:
            file_name = file_name.rsplit('.', 1)[0]
        standard_name = file_name.replace(' ', '_').lower()
        return standard_name

    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            gedcom_lines = f.readlines()
        current_dataset = None
        for line in gedcom_lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)
            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC']:
                    current_dataset.add_extractable_detail(tag, value)

        for dataset in self.gedcom_datasets:
            if dataset.get_extractable_NPFX():
                self.filter_pool.append(dataset)

# Moved extract_id outside of the class so it's a standalone function
def extract_id(record):
    id_start = record.find('@') + 1
    id_end = record.find('@', id_start)
    return record[id_start:id_end]

class GedcomDataset:
    def __init__(self, individual_id):
        self.individual_id = individual_id
        self.extractable_detail = {}
        self.anchor_gen1 = None
    def __init__(self, individual_id):
        self.individual_id = individual_id
        self.extractable_detail = {}
        self.anchor_gen1 = None  # Initialize this to None

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        return self.individual_id.strip('@')

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_anchor_gen1(self):
        name = self.extractable_detail.get('NAME', '')
        if '/' in name:
            first_name, last_name = name.split('/', 1)
            first_name = first_name.split(' ')[0]
            last_name = last_name.rstrip('/')
            self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
            return self.anchor_gen1
        else:
            print(f"Warning: No '/' found in NAME tag for individual {self.get_gen_person()}")
            return ''


# Your existing code for selecting a GEDCOM file
def select_gedcom_file():
    gedcom_files = glob.glob('*.ged')
    if not gedcom_files:
        print("No GEDCOM files found.")
        return None
    print("Automatically selecting the first GEDCOM file.")
    return gedcom_files[0]

gedcom_file_path = select_gedcom_file()
if gedcom_file_path:
    gedcom_instance = Parser()
    gedcom_instance.parse_file(gedcom_file_path)

    # Read the GEDCOM file and split it into individual and family records
    with open(gedcom_file_path, 'r') as file:
        data = file.read()
    data = data.split('\n0 ')
    records = {extract_id(record): record for record in data}  # Using standalone function

# Main execution code
gedcom_file_path = select_gedcom_file()
if gedcom_file_path:
    gedcom_instance = Gedcom(gedcom_file_path)
    gedcom_instance.parse_gedcom()

individuals = []
for dataset in gedcom_instance.gedcom_datasets:
    individual_id = dataset.get_gen_person()
    anchor_gen1 = dataset.get_anchor_gen1()
    individuals.append((anchor_gen1, individual_id))

print(f'Records tagged and filtered by NPFX: {len(individuals)}')


def find_parents(individual_id, generation, records):
    if individual_id not in records:
        return
    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]
    if famc_id not in records:
        return

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]
    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if mother_id and mother_id in records:
        mother_record = records[mother_id]
        mother_name = extract_name(mother_record)
    else:
        mother_name = None

    if father_id and father_id in records:
        father_record = records[father_id]
        father_name = extract_name(father_record)
    else:
        father_name = None

    if mother_name is not None and father_name is not None:
        parent_pair = father_name + "&" + mother_name

    if mother_id:
        find_parents(mother_id, generation + 1, records)

    if father_id:
        find_parents(father_id, generation + 1, records)

def find_distant_ancestors(individual_id, records, path=None, spouses=None):
    if path is None:
        path = [individual_id]
    if spouses is None:
        spouses = {}

    # Fetch the record for the individual_id
    record = records.get(individual_id, "")

    if not record:
        return [], spouses

    paths = []

    # Finding parents
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]

    if famc_id in records:
        fam_record = records[famc_id]
        wife_start = fam_record.find('1 WIFE @') + 8
        wife_end = fam_record.find('@', wife_start)
        mother_id = fam_record[wife_start:wife_end]

        husb_start = fam_record.find('1 HUSB @') + 8
        husb_end = fam_record.find('@', husb_start)
        father_id = fam_record[husb_start:husb_end]

        if father_id:
            new_path = list(path)
            paths.extend(find_distant_ancestors(father_id, records, new_path)[0])

        if mother_id:
            new_path = list(path)
            paths.extend(find_distant_ancestors(mother_id, records, new_path)[0])

    # Handling spouses
    fams_start = record.find('1 FAMS @') + 8
    fams_end = record.find('@', fams_start)
    fams_id = record[fams_start:fams_end]

    if fams_id in records:
        fam_record = records[fams_id]
        spouse_start = fam_record.find('1 WIFE @') + 8 if '1 HUSB @' in fam_record else fam_record.find('1 HUSB @') + 8
        spouse_end = fam_record.find('@', spouse_start)
        spouse_id = fam_record[spouse_start:spouse_end]
        spouses[individual_id] = spouse_id

    if spouses is not None and spouse_id:
        spouses[individual_id] = spouse_id

    if not paths:  # If no parents or spouses were found
        return [path], spouses

    return paths, spouses

#**********************************************************************************************************
individual_id_to_search = "I47881"
distant_ancestors_paths = find_distant_ancestors(individual_id_to_search, records)
print("Distant Ancestors Paths:", distant_ancestors_paths)

indexed_paths = {i: path for i, path in enumerate(distant_ancestors_paths)}

# Assuming `spouses` is a dictionary where the key is the individual ID and the value is the spouse ID
spouses = {}  # Populate this based on your function that finds distant ancestors

name_paths = []

for path in distant_ancestors_paths:  # Make sure distant_ancestors_paths is a list of IDs (strings or integers)
    name_path_with_spouse = []
    for id in path:
        if isinstance(id, (str, int)):
          individual_name = extract_name(records.get(id, ''))

        spouse_id = spouses.get(id, None)
        spouse_name = extract_name(records.get(spouse_id, '')) if spouse_id else "unknown"
        name_path_with_spouse.append(f"{individual_name}&{spouse_name}")
    name_paths.append(name_path_with_spouse)

# Check for the surname 'Yates' in the paths
for idx, name_path in enumerate(name_paths):
    has_yates = any("Yates" in name for name in name_path)
    if has_yates:
        print(f"Path index: {idx}, Contains Yates: {has_yates}, Names: {name_path}")

# Initialize an empty dictionary to store the scores of each path
path_scores = {}

# Loop through the paths and apply the scoring scheme
for idx, name_path in enumerate(name_paths):
    score = 0
    for generation, name in enumerate(name_path):
        if 'Yates' in name:
            score += 1 * (generation + 1)  # Add 1 to generation because index starts at 0
    # Store the score of this path
    path_scores[idx] = score

# Find the index of the path with the highest score
winning_path_index = max(path_scores, key=path_scores.get)
winning_path_score = path_scores[winning_path_index]
winning_path_names = name_paths[winning_path_index]

# Print the winning path to the console
print(f"High Branch score was {winning_path_score}.")    #The winning path is at index {winning_path_index}
#print(f"Names in the winning path: {winning_path_names}")

# Reversing the list for highest generation first
winning_path_names.reverse()

# Adding spouses (Replace 'unknown' with actual spouse names if you have them)
spouse_names = ['unknown'] * len(winning_path_names)  # Replace with real names or keep as 'unknown'

# Combine names and spouses in your specified format
formatted_names = [f"{name}&{spouse}" for name, spouse in zip(winning_path_names, spouse_names)]

# Join all the formatted names with '|'
final_output = "|".join(formatted_names)

print("Selected DNA Branch:", final_output)

visited_pairs = set()

def process_individual(individual_id, gedcom_instance):
    global generation_table
    generation_table = pd.DataFrame(columns=['Generation', 'Parent Pair'])
    global visited_pairs
    visited_pairs = set()

    find_parents(individual_id, 1, records)

    # Concatenate the value of the anchor_gen1 attribute to the beginning of the generation table
    for dataset in gedcom_instance.filter_pool:
        if dataset.get_gen_person() == individual_id:
            anchor_gen1 = dataset.get_anchor_gen1()
            generation_table.loc[0] = [1, anchor_gen1]
            break

    generation_table = generation_table.sort_values('Generation', ascending=False).reset_index(drop=True)

    individual_data = {}
    for dataset in gedcom_instance.filter_pool:
        if dataset.get_gen_person() == individual_id:
            individual_data['cM'] = dataset.get_extractable_cm()
            individual_data['Sort'] = dataset.get_extractable_sort()
            break

    individual_data['Parent Pairs A10'] = '|'.join([f"{row['Parent Pair']}" for index, row in generation_table[:10].iterrows()])

    print(f"Returning data for {individual_id}: {individual_data}")  # Debug print
    return individual_data

import pandas as pd

# Global variables
generation_table = None
visited_pairs = set()

# Load the CSV into Colab as a manual filter by user.
df = pd.read_csv('filtered_ids.csv')
filtered_ids = set(df['ID'])
filtered_records = {k: v for k, v in records.items() if k in filtered_ids}
filtered_individuals = [(last_name, individual_id) for last_name, individual_id in individuals if individual_id in filtered_ids]
print(f'User additional manual filter: {len(filtered_individuals)}')

# Run the process_individual function for each individual in the filtered_individuals list and create a combined DataFrame
combined_df_rows = []
for name, individual_id in filtered_individuals:
    individual_data = process_individual(individual_id, gedcom_instance)
    cm = individual_data['cM']
    sort = individual_data['Sort']
    parent_pairs_a10 = individual_data['Parent Pairs A10']
    most_distant_ancestor = generation_table.iloc[0]['Parent Pair']  # Get the most distant ancestor from the last row of generation_table
    combined_df_rows.append([individual_id, name, sort, cm, most_distant_ancestor, parent_pairs_a10])

combined_df = pd.DataFrame(combined_df_rows, columns=['ID#', 'Name', 'Match to', 'cM', 'Most Distant Ancestor', 'Ancestral Line A10'])

def create_hotlink(row):# Function to create hotlinks
    url_base = "https://yates.one-name.net/tng/verticalchart.php?personID="
    person_id = row['ID#']
    hotlink = f'<a href="{url_base}{person_id}&tree=tree1&parentset=0&display=vertical&generations=8" target="_blank">{person_id}</a>'
    return hotlink

combined_df['LUN#'] = combined_df.apply(lambda row: create_hotlink(row), axis=1) # Apply the hotlink function to create the 'LUN#' column
# Change the order of the columns
combined_df = combined_df[['ID#', 'Name', 'Match to', 'cM', 'Most Distant Ancestor', 'LUN#', 'Ancestral Line A10']]
# Adjust index to start from 1 instead of 0
combined_df.index = combined_df.index + 1
# Print all records from the DataFrame
print(combined_df)
# Export the combined_df DataFrame to an Excel file
combined_df.to_excel('/content/output.xlsx', index=False)


Automatically selecting the first GEDCOM file.
Automatically selecting the first GEDCOM file.
Records tagged and filtered by NPFX: 50776
Distant Ancestors Paths: ([['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881'], ['I47881']], {'I47881': 'I47881'})


TypeError: ignored

In [1]:
!pip install pandas
!pip install python-gedcom